# Projet Segmentation des clients avec la méthode RFM (Récence, Fréquence et Montant)

Récence : Le temps entre le nouvel achat et la dernière commande passée.  
Fréquence : Combien de fois le client à acheter un même produit.   
Montant : Sommes des dépenses par client.  

In [133]:
import pandas as pd
import numpy as np

In [134]:
df_customers = pd.read_csv('db_customers.csv')
df_customers

,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,payment_type,payment_installments,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,credit_card,1.0,18.12,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
1,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,voucher,1.0,2.00,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
2,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,voucher,1.0,18.59,4,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,housewares
3,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,boleto,1.0,141.46,4,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,perfumery
4,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,credit_card,3.0,179.12,5,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,auto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105378,9c5dedf39a927c1b2549525ed64a053c,delivered,2017-03-09 09:54:05,2017-03-17 15:08:01,credit_card,3.0,85.08,5,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,health_beauty
105379,63943bddc261676b46f01ca7ac2f7bd8,delivered,2018-02-06 12:58:58,2018-02-28 17:37:56,credit_card,3.0,195.00,4,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,baby
105380,83c1379a015df1e13d02aae0204711ab,delivered,2017-08-27 14:46:43,2017-09-21 11:24:17,credit_card,5.0,271.01,5,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,home_appliances_2
105381,11c177c8e97725db2631073c19f07b62,delivered,2018-01-08 21:28:27,2018-01-25 23:32:54,credit_card,4.0,441.16,2,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,computers_accessories


In [135]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105383 entries, 0 to 105382
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       105383 non-null  object 
 1   order_status                   105383 non-null  object 
 2   order_purchase_timestamp       105383 non-null  object 
 3   order_delivered_customer_date  102224 non-null  object 
 4   payment_type                   105382 non-null  object 
 5   payment_installments           105382 non-null  float64
 6   payment_value                  105382 non-null  float64
 7   review_score                   105383 non-null  int64  
 8   customer_unique_id             105383 non-null  object 
 9   customer_zip_code_prefix       105383 non-null  int64  
 10  customer_city                  105383 non-null  object 
 11  customer_state                 105383 non-null  object 
 12  product_category_name_english 

In [136]:
df_customers.isnull().sum()

order_id                            0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    3159
payment_type                        1
payment_installments                1
payment_value                       1
review_score                        0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
product_category_name_english    2410
dtype: int64

In [137]:
df_customers['customer_unique_id'].value_counts().unique()

array([54, 29, 26, 24, 23, 22, 19, 18, 17, 15, 14, 13, 12, 11, 10,  9,  8,
        7,  6,  5,  4,  3,  2,  1], dtype=int64)

In [138]:
df_customers['customer_zip_code_prefix'].value_counts().unique()

array([144, 140, 123, 116, 107, 105,  96,  95,  92,  90,  89,  88,  86,
        84,  83,  82,  80,  78,  77,  76,  75,  74,  73,  70,  69,  68,
        67,  66,  65,  64,  62,  61,  60,  59,  58,  57,  56,  55,  54,
        53,  52,  51,  50,  49,  48,  47,  46,  45,  44,  43,  42,  41,
        40,  39,  38,  37,  36,  35,  34,  33,  32,  31,  30,  29,  28,
        27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,  15,
        14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,   2,
         1], dtype=int64)

In [139]:
df_customers['order_purchase_timestamp'].value_counts().unique()

array([42, 29, 26, 24, 22, 19, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,
        5,  4,  3,  2,  1], dtype=int64)

In [140]:
np.unique('order_id')

array(['order_id'], dtype='<U8')

## Séparation en trois dataset pour résoudre les différentes questions de la RFM

### Dataset pour la Récence

Récence : Le temps entre le nouvel achat et la dernière commande passée. 

In [141]:
df_recence = df_customers[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'customer_unique_id']]

In [142]:
df_recence.dropna()

,order_id,order_purchase_timestamp,order_delivered_customer_date,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,7c396fd4830fd04220f754e42b4e5bff
1,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,7c396fd4830fd04220f754e42b4e5bff
2,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,7c396fd4830fd04220f754e42b4e5bff
3,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,af07308b275d755c9edb36a90c618231
4,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,3a653a41f6f9fc3d2a113cf8398680e8
...,...,...,...,...
105378,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,2017-03-17 15:08:01,6359f309b166b0196dbf7ad2ac62bb5a
105379,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,2018-02-28 17:37:56,da62f9e57a76d978d02ab5362c509660
105380,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,2017-09-21 11:24:17,737520a9aad80b3fbbdad19b66b37b30
105381,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,2018-01-25 23:32:54,5097a5312c8b157bb7be58ae360ef43c


In [143]:
import datetime as dt
#change type in date
df_recence['order_purchase_timestamp'] = pd.to_datetime(df_recence['order_purchase_timestamp'])
#stock la date max du dataset
today_date = df_recence['order_purchase_timestamp'].max()
#groupby
df_recence = df_recence.groupby("customer_unique_id").agg(max_date=('order_purchase_timestamp', "max"))
df_recence["recence"] = df_recence['max_date']-today_date

<ipython-input-143-b878ba80299e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recence['order_purchase_timestamp'] = pd.to_datetime(df_recence['order_purchase_timestamp'])


In [181]:
type(df_recence['recence'])

pandas.core.series.Series

In [145]:
#change type only time
df_recence['recence']= df_recence['recence'].astype("timedelta64[D]")

In [178]:
df_recence

,index,customer_unique_id,max_date,recence
0,0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,-161.0
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,-164.0
2,2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,-586.0
3,3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,-370.0
4,4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,-337.0
...,...,...,...,...
96091,96091,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08 21:00:36,-496.0
96092,96092,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10 20:07:56,-311.0
96093,96093,ffff371b4d645b6ecea244b27531430a,2017-02-07 15:49:16,-618.0
96094,96094,ffff5962728ec6157033ef9805bacc48,2018-05-02 15:17:41,-169.0


### Dataset pour la Fréquence

Fréquence : Combien de fois le client à acheter un même produit.

In [147]:
df_frequence = df_customers[['order_id', 'customer_unique_id']]
df_frequence

,order_id,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
1,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
2,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
3,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231
4,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8
...,...,...
105378,9c5dedf39a927c1b2549525ed64a053c,6359f309b166b0196dbf7ad2ac62bb5a
105379,63943bddc261676b46f01ca7ac2f7bd8,da62f9e57a76d978d02ab5362c509660
105380,83c1379a015df1e13d02aae0204711ab,737520a9aad80b3fbbdad19b66b37b30
105381,11c177c8e97725db2631073c19f07b62,5097a5312c8b157bb7be58ae360ef43c


In [148]:
df_frequence = df_frequence.groupby("customer_unique_id").agg(count_products=('order_id', "count"))

In [165]:
df_frequence= df_frequence.reset_index()

In [171]:
df_frequence

,index,customer_unique_id,count_products
0,0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,2,0000f46a3911fa3c0805444483337064,1
3,3,0000f6ccb0745a6a4b88665a16c9f078,1
4,4,0004aac84e0df4da2b147fca70cf8255,1
...,...,...,...
96091,96091,fffcf5a5ff07b0908bd4e2dbc735a684,1
96092,96092,fffea47cd6d3cc0a88bd621562a9d061,1
96093,96093,ffff371b4d645b6ecea244b27531430a,1
96094,96094,ffff5962728ec6157033ef9805bacc48,1


In [183]:
df_frequence['count_products'].value_counts()

1     89630
2      5130
3       794
4       283
5        84
6        72
7        32
8        16
9        12
10        8
11        7
12        6
13        4
14        3
15        3
24        3
19        2
17        1
18        1
22        1
23        1
26        1
29        1
54        1
Name: count_products, dtype: int64

### Dataset pour le Montant

Montant : Sommes des dépenses par client.

In [184]:
df_montant = df_customers[['customer_unique_id', 'payment_value']]
df_montant

,customer_unique_id,payment_value
0,7c396fd4830fd04220f754e42b4e5bff,18.12
1,7c396fd4830fd04220f754e42b4e5bff,2.00
2,7c396fd4830fd04220f754e42b4e5bff,18.59
3,af07308b275d755c9edb36a90c618231,141.46
4,3a653a41f6f9fc3d2a113cf8398680e8,179.12
...,...,...
105378,6359f309b166b0196dbf7ad2ac62bb5a,85.08
105379,da62f9e57a76d978d02ab5362c509660,195.00
105380,737520a9aad80b3fbbdad19b66b37b30,271.01
105381,5097a5312c8b157bb7be58ae360ef43c,441.16


In [185]:
df_montant = df_montant.groupby('customer_unique_id').agg(montant = ("payment_value","sum"))

In [186]:
df_montant

,montant
customer_unique_id,
0000366f3b9a7992bf8c76cfdf3221e2,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,27.19
0000f46a3911fa3c0805444483337064,86.22
0000f6ccb0745a6a4b88665a16c9f078,43.62
0004aac84e0df4da2b147fca70cf8255,196.89
...,...
fffcf5a5ff07b0908bd4e2dbc735a684,2067.42
fffea47cd6d3cc0a88bd621562a9d061,84.58
ffff371b4d645b6ecea244b27531430a,112.46
